In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import sklearn.metrics as metrics
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv

In [6]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='Cora', name='Cora')

Processing...
Done!


# Question 1: Testing parameters in GCN

## 1. Parameter hidden layer dimensions: 2, 4, 8, 16, 64, 512

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv

class Net(torch.nn.Module):
    def __init__(self, hidden_size):
        super(Net, self).__init__()
        # you can change convolutions here
        #self.conv1 = SAGEConv(dataset.num_node_features, 16)
        #self.conv2 = SAGEConv(16, dataset.num_classes)
        self.conv1 = GCNConv(dataset.num_node_features, hidden_size)
        self.conv2 = GCNConv(hidden_size, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
hidden_layer_dimensions = [2,4,8,16,64,512]

for dimension in hidden_layer_dimensions: # Looping through a set of different hidden layer sizes

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = Net(hidden_size = dimension).to(device)
  data = dataset[0].to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

  model.train()
  print(f"####Training GCN with a {dimension} hidden layer dimension...####")
  for epoch in range(100):
      optimizer.zero_grad()
      out = model(data)
      loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()

      _, pred = model(data).max(dim=1)
      correct = float (pred[data.train_mask].eq(data.y[data.train_mask]).sum().item())
      acc = correct / data.train_mask.sum().item()
      print('Epoch: %d, Accuracy: %.4f'%(epoch,acc))

  _, pred = model(data).max(dim=1)
  correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
  acc = correct / data.test_mask.sum().item()
  print('####Test Accuracy with a', dimension, 'hidden layer dimension: {:.4f}####'.format(acc))

####Training GCN with 2 hidden layers...####
Epoch: 0, Accuracy: 0.2357
Epoch: 1, Accuracy: 0.2929
Epoch: 2, Accuracy: 0.2500
Epoch: 3, Accuracy: 0.2571
Epoch: 4, Accuracy: 0.2714
Epoch: 5, Accuracy: 0.2786
Epoch: 6, Accuracy: 0.3071
Epoch: 7, Accuracy: 0.3286
Epoch: 8, Accuracy: 0.3286
Epoch: 9, Accuracy: 0.3286
Epoch: 10, Accuracy: 0.3357
Epoch: 11, Accuracy: 0.3429
Epoch: 12, Accuracy: 0.3429
Epoch: 13, Accuracy: 0.3500
Epoch: 14, Accuracy: 0.3571
Epoch: 15, Accuracy: 0.3857
Epoch: 16, Accuracy: 0.4500
Epoch: 17, Accuracy: 0.5214
Epoch: 18, Accuracy: 0.5286
Epoch: 19, Accuracy: 0.5571
Epoch: 20, Accuracy: 0.5714
Epoch: 21, Accuracy: 0.5643
Epoch: 22, Accuracy: 0.5714
Epoch: 23, Accuracy: 0.5857
Epoch: 24, Accuracy: 0.5929
Epoch: 25, Accuracy: 0.6071
Epoch: 26, Accuracy: 0.6071
Epoch: 27, Accuracy: 0.6214
Epoch: 28, Accuracy: 0.6500
Epoch: 29, Accuracy: 0.6786
Epoch: 30, Accuracy: 0.7000
Epoch: 31, Accuracy: 0.7500
Epoch: 32, Accuracy: 0.7571
Epoch: 33, Accuracy: 0.7643
Epoch: 34, Ac

## 2. Parameter number of hidden layers: 1, 2, 4, 8, 16

## Manually trying all different number of hidden layers



## With 1 hidden layer

In [32]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # you can change convolutions here
        # 1 hidden layer
        self.conv1 = GCNConv(dataset.num_node_features, dataset.num_classes)
        #self.conv1 = SAGEConv(dataset.num_node_features, 16)
        #self.conv2 = SAGEConv(16, dataset.num_classes)
        # self.conv1 = GCNConv(dataset.num_node_features, 16)
        # self.conv2 = GCNConv(16, dataset.num_classes)


    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        # x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    _, pred = model(data).max(dim=1)
    correct = float (pred[data.train_mask].eq(data.y[data.train_mask]).sum().item())
    acc = correct / data.train_mask.sum().item()
    print('Epoch: %d, Accuracy: %.4f'%(epoch,acc))

_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('####Test Accuracy with 1 hidden layer: {:.4f}####'.format(acc))

Epoch: 0, Accuracy: 0.4857
Epoch: 1, Accuracy: 0.6000
Epoch: 2, Accuracy: 0.6786
Epoch: 3, Accuracy: 0.8143
Epoch: 4, Accuracy: 0.8643
Epoch: 5, Accuracy: 0.9000
Epoch: 6, Accuracy: 0.9357
Epoch: 7, Accuracy: 0.9500
Epoch: 8, Accuracy: 0.9714
Epoch: 9, Accuracy: 0.9714
Epoch: 10, Accuracy: 0.9714
Epoch: 11, Accuracy: 0.9786
Epoch: 12, Accuracy: 0.9786
Epoch: 13, Accuracy: 0.9857
Epoch: 14, Accuracy: 0.9857
Epoch: 15, Accuracy: 0.9857
Epoch: 16, Accuracy: 0.9929
Epoch: 17, Accuracy: 0.9929
Epoch: 18, Accuracy: 0.9929
Epoch: 19, Accuracy: 0.9929
Epoch: 20, Accuracy: 0.9929
Epoch: 21, Accuracy: 0.9929
Epoch: 22, Accuracy: 0.9929
Epoch: 23, Accuracy: 0.9929
Epoch: 24, Accuracy: 1.0000
Epoch: 25, Accuracy: 1.0000
Epoch: 26, Accuracy: 1.0000
Epoch: 27, Accuracy: 1.0000
Epoch: 28, Accuracy: 1.0000
Epoch: 29, Accuracy: 1.0000
Epoch: 30, Accuracy: 1.0000
Epoch: 31, Accuracy: 1.0000
Epoch: 32, Accuracy: 1.0000
Epoch: 33, Accuracy: 1.0000
Epoch: 34, Accuracy: 1.0000
Epoch: 35, Accuracy: 1.0000
Ep

## With 2 hidden layers

In [31]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # you can change convolutions here
        #self.conv1 = SAGEConv(dataset.num_node_features, 16)
        #self.conv2 = SAGEConv(16, dataset.num_classes)
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    _, pred = model(data).max(dim=1)
    correct = float (pred[data.train_mask].eq(data.y[data.train_mask]).sum().item())
    acc = correct / data.train_mask.sum().item()
    print('Epoch: %d, Accuracy: %.4f'%(epoch,acc))

_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('####Test Accuracy with 2 hidden layers: {:.4f}####'.format(acc))

Epoch: 0, Accuracy: 0.6000
Epoch: 1, Accuracy: 0.7643
Epoch: 2, Accuracy: 0.8143
Epoch: 3, Accuracy: 0.8500
Epoch: 4, Accuracy: 0.8714
Epoch: 5, Accuracy: 0.9000
Epoch: 6, Accuracy: 0.9143
Epoch: 7, Accuracy: 0.9500
Epoch: 8, Accuracy: 0.9643
Epoch: 9, Accuracy: 0.9643
Epoch: 10, Accuracy: 0.9786
Epoch: 11, Accuracy: 0.9857
Epoch: 12, Accuracy: 0.9857
Epoch: 13, Accuracy: 0.9857
Epoch: 14, Accuracy: 0.9857
Epoch: 15, Accuracy: 0.9857
Epoch: 16, Accuracy: 0.9929
Epoch: 17, Accuracy: 1.0000
Epoch: 18, Accuracy: 1.0000
Epoch: 19, Accuracy: 1.0000
Epoch: 20, Accuracy: 1.0000
Epoch: 21, Accuracy: 1.0000
Epoch: 22, Accuracy: 1.0000
Epoch: 23, Accuracy: 1.0000
Epoch: 24, Accuracy: 1.0000
Epoch: 25, Accuracy: 1.0000
Epoch: 26, Accuracy: 1.0000
Epoch: 27, Accuracy: 1.0000
Epoch: 28, Accuracy: 1.0000
Epoch: 29, Accuracy: 1.0000
Epoch: 30, Accuracy: 1.0000
Epoch: 31, Accuracy: 1.0000
Epoch: 32, Accuracy: 1.0000
Epoch: 33, Accuracy: 1.0000
Epoch: 34, Accuracy: 1.0000
Epoch: 35, Accuracy: 1.0000
Ep

## With 4 hidden layers

In [27]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # you can change convolutions here
        #self.conv1 = SAGEConv(dataset.num_node_features, 16)
        #self.conv2 = SAGEConv(16, dataset.num_classes)
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16,16)
        self.conv3 = GCNConv(16,16)
        self.conv4 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.conv4(x, edge_index)

        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    _, pred = model(data).max(dim=1)
    correct = float (pred[data.train_mask].eq(data.y[data.train_mask]).sum().item())
    acc = correct / data.train_mask.sum().item()
    print('Epoch: %d, Accuracy: %.4f'%(epoch,acc))

_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('####Test Accuracy with 4 hidden layers: {:.4f}####'.format(acc))

Epoch: 0, Accuracy: 0.2857
Epoch: 1, Accuracy: 0.4500
Epoch: 2, Accuracy: 0.4429
Epoch: 3, Accuracy: 0.4643
Epoch: 4, Accuracy: 0.5214
Epoch: 5, Accuracy: 0.5929
Epoch: 6, Accuracy: 0.6643
Epoch: 7, Accuracy: 0.6786
Epoch: 8, Accuracy: 0.6786
Epoch: 9, Accuracy: 0.7000
Epoch: 10, Accuracy: 0.7071
Epoch: 11, Accuracy: 0.7071
Epoch: 12, Accuracy: 0.7071
Epoch: 13, Accuracy: 0.7143
Epoch: 14, Accuracy: 0.7357
Epoch: 15, Accuracy: 0.8143
Epoch: 16, Accuracy: 0.8786
Epoch: 17, Accuracy: 0.9643
Epoch: 18, Accuracy: 0.9786
Epoch: 19, Accuracy: 0.9786
Epoch: 20, Accuracy: 0.9929
Epoch: 21, Accuracy: 0.9929
Epoch: 22, Accuracy: 0.9929
Epoch: 23, Accuracy: 0.9929
Epoch: 24, Accuracy: 1.0000
Epoch: 25, Accuracy: 1.0000
Epoch: 26, Accuracy: 1.0000
Epoch: 27, Accuracy: 1.0000
Epoch: 28, Accuracy: 1.0000
Epoch: 29, Accuracy: 1.0000
Epoch: 30, Accuracy: 1.0000
Epoch: 31, Accuracy: 1.0000
Epoch: 32, Accuracy: 1.0000
Epoch: 33, Accuracy: 1.0000
Epoch: 34, Accuracy: 1.0000
Epoch: 35, Accuracy: 1.0000
Ep

## With 8 hidden layers

In [28]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # you can change convolutions here
        #self.conv1 = SAGEConv(dataset.num_node_features, 16)
        #self.conv2 = SAGEConv(16, dataset.num_classes)
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16,16)
        self.conv3 = GCNConv(16,16)
        self.conv4 = GCNConv(16,16)
        self.conv5 = GCNConv(16,16)
        self.conv6 = GCNConv(16,16)
        self.conv7 = GCNConv(16,16)
        self.conv8 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.conv4(x, edge_index)
        x = F.relu(x)
        x = self.conv5(x, edge_index)
        x = F.relu(x)
        x = self.conv6(x, edge_index)
        x = F.relu(x)
        x = self.conv7(x, edge_index)
        x = F.relu(x)

        x = self.conv8(x, edge_index)

        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    _, pred = model(data).max(dim=1)
    correct = float (pred[data.train_mask].eq(data.y[data.train_mask]).sum().item())
    acc = correct / data.train_mask.sum().item()
    print('Epoch: %d, Accuracy: %.4f'%(epoch,acc))

_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('####Test Accuracy with 8 hidden layers: {:.4f}####'.format(acc))

Epoch: 0, Accuracy: 0.1429
Epoch: 1, Accuracy: 0.1500
Epoch: 2, Accuracy: 0.1571
Epoch: 3, Accuracy: 0.1714
Epoch: 4, Accuracy: 0.1786
Epoch: 5, Accuracy: 0.2143
Epoch: 6, Accuracy: 0.2500
Epoch: 7, Accuracy: 0.2571
Epoch: 8, Accuracy: 0.2643
Epoch: 9, Accuracy: 0.2857
Epoch: 10, Accuracy: 0.2786
Epoch: 11, Accuracy: 0.3500
Epoch: 12, Accuracy: 0.3214
Epoch: 13, Accuracy: 0.3357
Epoch: 14, Accuracy: 0.4429
Epoch: 15, Accuracy: 0.3286
Epoch: 16, Accuracy: 0.4357
Epoch: 17, Accuracy: 0.5071
Epoch: 18, Accuracy: 0.4286
Epoch: 19, Accuracy: 0.5429
Epoch: 20, Accuracy: 0.5143
Epoch: 21, Accuracy: 0.5357
Epoch: 22, Accuracy: 0.5071
Epoch: 23, Accuracy: 0.5929
Epoch: 24, Accuracy: 0.6214
Epoch: 25, Accuracy: 0.5714
Epoch: 26, Accuracy: 0.6000
Epoch: 27, Accuracy: 0.6143
Epoch: 28, Accuracy: 0.6357
Epoch: 29, Accuracy: 0.7214
Epoch: 30, Accuracy: 0.6786
Epoch: 31, Accuracy: 0.7857
Epoch: 32, Accuracy: 0.7429
Epoch: 33, Accuracy: 0.7786
Epoch: 34, Accuracy: 0.8214
Epoch: 35, Accuracy: 0.8143
Ep

## With 16 hidden layers

In [30]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # you can change convolutions here
        #self.conv1 = SAGEConv(dataset.num_node_features, 16)
        #self.conv2 = SAGEConv(16, dataset.num_classes)
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16,16)
        self.conv3 = GCNConv(16,16)
        self.conv4 = GCNConv(16,16)
        self.conv5 = GCNConv(16,16)
        self.conv6 = GCNConv(16,16)
        self.conv7 = GCNConv(16,16)
        self.conv8 = GCNConv(16,16)
        self.conv9 = GCNConv(16,16)
        self.conv10 = GCNConv(16,16)
        self.conv11 = GCNConv(16,16)
        self.conv12 = GCNConv(16,16)
        self.conv13 = GCNConv(16,16)
        self.conv14 = GCNConv(16,16)
        self.conv15 = GCNConv(16,16)
        self.conv16 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.conv4(x, edge_index)
        x = F.relu(x)
        x = self.conv5(x, edge_index)
        x = F.relu(x)
        x = self.conv6(x, edge_index)
        x = F.relu(x)
        x = self.conv7(x, edge_index)
        x = F.relu(x)
        x = self.conv8(x, edge_index)
        x = F.relu(x)
        x = self.conv9(x, edge_index)
        x = F.relu(x)
        x = self.conv10(x, edge_index)
        x = F.relu(x)
        x = self.conv11(x, edge_index)
        x = F.relu(x)
        x = self.conv12(x, edge_index)
        x = F.relu(x)
        x = self.conv13(x, edge_index)
        x = F.relu(x)
        x = self.conv14(x, edge_index)
        x = F.relu(x)
        x = self.conv15(x, edge_index)
        x = F.relu(x)
        x = self.conv16(x, edge_index)

        return F.log_softmax(x, dim=1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    _, pred = model(data).max(dim=1)
    correct = float (pred[data.train_mask].eq(data.y[data.train_mask]).sum().item())
    acc = correct / data.train_mask.sum().item()
    print('Epoch: %d, Accuracy: %.4f'%(epoch,acc))

_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('####Test Accuracy with 16 hidden layers: {:.4f}####'.format(acc))

Epoch: 0, Accuracy: 0.1500
Epoch: 1, Accuracy: 0.1429
Epoch: 2, Accuracy: 0.1571
Epoch: 3, Accuracy: 0.1786
Epoch: 4, Accuracy: 0.1571
Epoch: 5, Accuracy: 0.1357
Epoch: 6, Accuracy: 0.1500
Epoch: 7, Accuracy: 0.1571
Epoch: 8, Accuracy: 0.1214
Epoch: 9, Accuracy: 0.1429
Epoch: 10, Accuracy: 0.1286
Epoch: 11, Accuracy: 0.1286
Epoch: 12, Accuracy: 0.1286
Epoch: 13, Accuracy: 0.1357
Epoch: 14, Accuracy: 0.1429
Epoch: 15, Accuracy: 0.1429
Epoch: 16, Accuracy: 0.1500
Epoch: 17, Accuracy: 0.1500
Epoch: 18, Accuracy: 0.1429
Epoch: 19, Accuracy: 0.1429
Epoch: 20, Accuracy: 0.1357
Epoch: 21, Accuracy: 0.1571
Epoch: 22, Accuracy: 0.1571
Epoch: 23, Accuracy: 0.1643
Epoch: 24, Accuracy: 0.1571
Epoch: 25, Accuracy: 0.1643
Epoch: 26, Accuracy: 0.1643
Epoch: 27, Accuracy: 0.1643
Epoch: 28, Accuracy: 0.1714
Epoch: 29, Accuracy: 0.1714
Epoch: 30, Accuracy: 0.1714
Epoch: 31, Accuracy: 0.1857
Epoch: 32, Accuracy: 0.1857
Epoch: 33, Accuracy: 0.1857
Epoch: 34, Accuracy: 0.1857
Epoch: 35, Accuracy: 0.1643
Ep

## 3. Parameter learning rate: 1, .5, .1, .01, .0001

In [33]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # you can change convolutions here
        #self.conv1 = SAGEConv(dataset.num_node_features, 16)
        #self.conv2 = SAGEConv(16, dataset.num_classes)
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
learning_rates = [1, .5, .1, .01, .0001]

for rate in learning_rates:
  optimizer = torch.optim.Adam(model.parameters(), lr=rate, weight_decay=5e-4)
  model.train()
  print(f"####Training GCN with a learning rate of {rate}...####")
  for epoch in range(100):
      optimizer.zero_grad()
      out = model(data)
      loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
      loss.backward()
      optimizer.step()

      _, pred = model(data).max(dim=1)
      correct = float (pred[data.train_mask].eq(data.y[data.train_mask]).sum().item())
      acc = correct / data.train_mask.sum().item()
      print('Epoch: %d, Accuracy: %.4f'%(epoch,acc))


  _, pred = model(data).max(dim=1)
  correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
  acc = correct / data.test_mask.sum().item()
  print('####Test Accuracy with a learning rate of', rate, 'is: {:.4f}####'.format(acc))

####Training GCN with a learning rate of 1...####
Epoch: 0, Accuracy: 0.1643
Epoch: 1, Accuracy: 0.2714
Epoch: 2, Accuracy: 0.4286
Epoch: 3, Accuracy: 0.3929
Epoch: 4, Accuracy: 0.4500
Epoch: 5, Accuracy: 0.5357
Epoch: 6, Accuracy: 0.5714
Epoch: 7, Accuracy: 0.6286
Epoch: 8, Accuracy: 0.6071
Epoch: 9, Accuracy: 0.6071
Epoch: 10, Accuracy: 0.6429
Epoch: 11, Accuracy: 0.6714
Epoch: 12, Accuracy: 0.6786
Epoch: 13, Accuracy: 0.5786
Epoch: 14, Accuracy: 0.6929
Epoch: 15, Accuracy: 0.6857
Epoch: 16, Accuracy: 0.6786
Epoch: 17, Accuracy: 0.6786
Epoch: 18, Accuracy: 0.6857
Epoch: 19, Accuracy: 0.6714
Epoch: 20, Accuracy: 0.6786
Epoch: 21, Accuracy: 0.6857
Epoch: 22, Accuracy: 0.6286
Epoch: 23, Accuracy: 0.6929
Epoch: 24, Accuracy: 0.7000
Epoch: 25, Accuracy: 0.7286
Epoch: 26, Accuracy: 0.7143
Epoch: 27, Accuracy: 0.7286
Epoch: 28, Accuracy: 0.7286
Epoch: 29, Accuracy: 0.7214
Epoch: 30, Accuracy: 0.7286
Epoch: 31, Accuracy: 0.7286
Epoch: 32, Accuracy: 0.7071
Epoch: 33, Accuracy: 0.7286
Epoch: 3